# Imports

In [49]:
import pandas as pd
import numpy as np
import nltk
import re
import string
# nltk.download("all")
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from sklearn.utils.class_weight import compute_class_weight

# Reading the Dataset

In [2]:
data = pd.read_csv("Combined_News_DJIA.csv")
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [3]:
# print(data.iterrows())

In [4]:
data.columns[2:][0]

'Top1'

# Converting the data into X and Y

In [10]:
x = []
y = []
for i in data.iterrows():
    text = ""
    y.append(i[1]['Label'])
    for j in data.columns[2:]:
        if type(i[1][j]) != str and type(i[1][j]) != object:
            continue
        text += i[1][j][1:] + " "
    x.append(text)

In [11]:
x = np.array(x)
y = np.array(y)
print(x.shape, y.shape)
print(x[0])
print(y[0])

(1989,) (1989,)
"Georgia 'downs two Russian warplanes' as countries move to brink of war" 'BREAKING: Musharraf to be impeached.' 'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)' 'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire' "Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing" '150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.' "Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side" "The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." 'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]' 'Did the U.S. Prep Georgia for War with Russia?' 'Rice Gives G

# Preprocessing Text

In [12]:
wnl = WordNetLemmatizer()

In [14]:
print(x[0][1:])

Georgia 'downs two Russian warplanes' as countries move to brink of war" 'BREAKING: Musharraf to be impeached.' 'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)' 'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire' "Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing" '150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.' "Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side" "The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." 'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]' 'Did the U.S. Prep Georgia for War with Russia?' 'Rice Gives Green Light for Is

In [15]:
for i in tqdm(range(len(x))):
    text = x[i][1:]
    text = text.lower()
    text = [j for j in text if j not in string.punctuation]
    text = ''.join(text)
    text = re.sub(r"[^a-zA-Z]", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = [wnl.lemmatize(word) for word in text.split() if word not in stopwords.words("english")]
    text = ' '.join(text)
    x[i] = text

100%|██████████| 1989/1989 [02:45<00:00, 12.03it/s]


In [16]:
print(x[0])

georgia down two russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick three year old raped nothing russian tank entered south ossetia whilst georgia shoot two russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several hundred people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war iii start toda

# Bag of Words

In [50]:
bow = CountVectorizer(max_features=50000)
bow_corpus = bow.fit_transform(x).toarray()
x1 = bow_corpus
y1 = y
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.20, random_state=42, shuffle=True)
class_weights1 = compute_class_weight('balanced', classes=np.unique(y_train1), y=y_train1)

In [52]:
print(class_weights1)

[1.0564409  0.94928401]


In [53]:
model_bow1 = SVC(class_weight=dict(zip(np.unique(y_train1), class_weights1)))
model_bow2 = LogisticRegression(class_weight=dict(zip(np.unique(y_train1), class_weights1)))
model_bow3 = DecisionTreeClassifier(class_weight=dict(zip(np.unique(y_train1), class_weights1)))
model_bow4 = RandomForestClassifier(class_weight=dict(zip(np.unique(y_train1), class_weights1)))

models1 = [model_bow1, model_bow2, model_bow3, model_bow4]

models1 = Parallel(n_jobs=-1)(delayed(models1[i].fit)(x_train1, y_train1) for i in range(len(models1)))
print(models1)

[SVC(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), LogisticRegression(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), DecisionTreeClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394}), RandomForestClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394})]


In [54]:
predictions1 = Parallel(n_jobs=-1)(delayed(models1[i].predict)(x_test1) for i in range(len(models1)))
print("Accuracy SVM: ", accuracy_score(predictions1[0], y_test1))
print("Accuracy LR: ", accuracy_score(predictions1[1], y_test1))
print("Accuracy DT: ", accuracy_score(predictions1[2], y_test1))
print("Accuracy RF: ", accuracy_score(predictions1[3], y_test1))

Accuracy SVM:  0.535175879396985
Accuracy MNB:  0.5050251256281407
Accuracy DT:  0.49246231155778897
Accuracy RF:  0.5150753768844221


# TFIDF

In [55]:
tfidf = TfidfVectorizer(max_features=50000)
tfidf_corpus = tfidf.fit_transform(x).toarray()
x2 = tfidf_corpus
y2 = y
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.20, random_state=42, shuffle=True)
class_weights2 = compute_class_weight('balanced', classes=np.unique(y_train2), y=y_train2)

In [57]:
model_tfidf1 = SVC(class_weight=dict(zip(np.unique(y_train2), class_weights2)))
model_tfidf2 = LogisticRegression(class_weight=dict(zip(np.unique(y_train2), class_weights2)))
model_tfidf3 = DecisionTreeClassifier(class_weight=dict(zip(np.unique(y_train2), class_weights2)))
model_tfidf4 = RandomForestClassifier(class_weight=dict(zip(np.unique(y_train2), class_weights2)))

models2 = [model_tfidf1, model_tfidf2, model_tfidf3, model_tfidf4]

models2 = Parallel(n_jobs=-1)(delayed(models2[i].fit)(x_train2, y_train2) for i in range(len(models2)))
print(models2)

[SVC(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), LogisticRegression(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), DecisionTreeClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394}), RandomForestClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394})]


In [58]:
predictions2 = Parallel(n_jobs=-1)(delayed(models2[i].predict)(x_test2) for i in range(len(models2)))
print("Accuracy SVM: ", accuracy_score(predictions2[0], y_test2))
print("Accuracy LR: ", accuracy_score(predictions2[1], y_test2))
print("Accuracy DT: ", accuracy_score(predictions2[2], y_test2))
print("Accuracy RF: ", accuracy_score(predictions2[3], y_test2))

Accuracy SVM:  0.535175879396985
Accuracy LR:  0.5100502512562815
Accuracy DT:  0.4723618090452261
Accuracy RF:  0.5527638190954773


# Word2Vec

In [59]:
w2v_corpus = []
# print(type(x[0]))
for i in x:
    words = []
    for word in i.split():
        words.append(word)
    w2v_corpus.append(words)
print(len(w2v_corpus))
print(w2v_corpus[0])

1989
['georgia', 'down', 'two', 'russian', 'warplane', 'country', 'move', 'brink', 'war', 'breaking', 'musharraf', 'impeached', 'russia', 'today', 'column', 'troop', 'roll', 'south', 'ossetia', 'footage', 'fighting', 'youtube', 'russian', 'tank', 'moving', 'towards', 'capital', 'south', 'ossetia', 'reportedly', 'completely', 'destroyed', 'georgian', 'artillery', 'fire', 'afghan', 'child', 'raped', 'impunity', 'un', 'official', 'say', 'sick', 'three', 'year', 'old', 'raped', 'nothing', 'russian', 'tank', 'entered', 'south', 'ossetia', 'whilst', 'georgia', 'shoot', 'two', 'russian', 'jet', 'breaking', 'georgia', 'invades', 'south', 'ossetia', 'russia', 'warned', 'would', 'intervene', 'so', 'side', 'enemy', 'combatent', 'trial', 'nothing', 'sham', 'salim', 'haman', 'sentenced', 'year', 'kept', 'longer', 'anyway', 'feel', 'like', 'georgian', 'troop', 'retreat', 'osettain', 'capital', 'presumably', 'leaving', 'several', 'hundred', 'people', 'killed', 'video', 'u', 'prep', 'georgia', 'war', 

In [60]:
w2v = Word2Vec(w2v_corpus, vector_size = 500)
# w2v.wv['georgia']

In [61]:
document_embeddings = []
for paragraph in tqdm(x):
    tokenized_paragraph = nltk.word_tokenize(paragraph.lower())
    paragraph_embedding = np.mean([w2v.wv[word] for word in tokenized_paragraph if word in w2v.wv], axis=0)
    document_embeddings.append(paragraph_embedding)

x_train3, x_test3, y_train3, y_test3 = train_test_split(document_embeddings, y, test_size=0.2, random_state=42)

  0%|          | 0/1989 [00:00<?, ?it/s]100%|██████████| 1989/1989 [00:03<00:00, 499.61it/s]


In [62]:
# print(len(document_embeddings[0]))
class_weights3 = compute_class_weight('balanced', classes=np.unique(y_train3), y=y_train3)

In [63]:
model_w2v1 = SVC(class_weight=dict(zip(np.unique(y_train3), class_weights3)))
model_w2v2 = LogisticRegression(class_weight=dict(zip(np.unique(y_train3), class_weights3)))
model_w2v3 = DecisionTreeClassifier(class_weight=dict(zip(np.unique(y_train3), class_weights3)))
model_w2v4 = RandomForestClassifier(class_weight=dict(zip(np.unique(y_train3), class_weights3)))

models3 = [model_w2v1, model_w2v2, model_w2v3, model_w2v4]

models3 = Parallel(n_jobs=-1)(delayed(models3[i].fit)(x_train3, y_train3) for i in range(len(models3)))
print(models3)

[SVC(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), LogisticRegression(class_weight={0: 1.0564409030544488, 1: 0.9492840095465394}), DecisionTreeClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394}), RandomForestClassifier(class_weight={0: 1.0564409030544488,
                                     1: 0.9492840095465394})]


In [64]:
predictions3 = Parallel(n_jobs=-1)(delayed(models3[i].predict)(x_test3) for i in range(len(models3)))
print("Accuracy SVM: ", accuracy_score(predictions3[0], y_test3))
print("Accuracy MNB: ", accuracy_score(predictions3[1], y_test3))
print("Accuracy DT: ", accuracy_score(predictions3[2], y_test3))
print("Accuracy RF: ", accuracy_score(predictions3[3], y_test3))

Accuracy SVM:  0.5050251256281407
Accuracy MNB:  0.5201005025125628
Accuracy DT:  0.48743718592964824
Accuracy RF:  0.5577889447236181
